In [293]:
import sqlite3
print(sqlite3.sqlite_version_info)

(3, 42, 0)


In [294]:
import pandas as pd
from datetime import datetime
from IPython.display import display, HTML

In [295]:
dbname = "music_streaming_1.db"

def printSqlResults(cursor, tbl):
  df=pd.DataFrame(cursor.fetchall(), columns=[i[0] for i in cursor.description])
  display(HTML("<b><font color=Green>"+tbl+"</font></b>"+df.to_html(index=False)))
  return df

def runSql(caption, query):
  conn = sqlite3.connect(dbname)
  cursor=conn.cursor()
  cursor.execute(query)
  result = printSqlResults(cursor,caption)
  conn.close()
  return result

def write_to_table(df,table_name):
  conn = sqlite3.connect(dbname)
  try:
    df.to_sql(table_name, conn, if_exists='append', index=False)
    conn.close()
    return True
  except:
    return False
  

In [296]:
conn = sqlite3.connect(dbname)
cursor = conn.cursor()

cursor.execute("""
      CREATE TABLE IF NOT EXISTS Users (
        user_id INTEGER PRIMARY KEY,
        name VARCHAR(100) NOT NULL,
        email VARCHAR(100) NOT NULL UNIQUE
      );
""")

cursor.execute("""
  CREATE TABLE IF NOT EXISTS Songs (
    song_id INTEGER PRIMARY KEY,
    title VARCHAR(100) NOT NULL,
    artist VARCHAR(100) NOT NULL,
    genre VARCHAR(100)
  );
"""
)

cursor.execute("""
    CREATE TABLE IF NOT EXISTS Listens (
      listen_id INTEGER PRIMARY KEY,
      user_id INTEGER NOT NULL,
      song_id INTEGER NOT NULL,
      rating FLOAT,
      listen_time TIMESTAMP,
      FOREIGN KEY (user_id) REFERENCES Users(user_id),
      FOREIGN KEY (song_id) REFERENCES Songs(song_id)
    );
""")

cursor.execute("""
  CREATE TABLE IF NOT EXISTS Recommendations (
      recommendation_id INTEGER NOT NULL,
      recommendation_time TIMESTAMP,
      user_id INTEGER NOT NULL,
      song_id INTEGER NOT NULL,
      FOREIGN KEY (user_id) REFERENCES Users(user_id),
      FOREIGN KEY (song_id) REFERENCES Songs(song_id)
  );
""")

conn.commit()
conn.close()

In [297]:
conn = sqlite3.connect(dbname)
sqlite3.enable_callback_tracebacks(True)

cursor=conn.cursor()
cursor.execute("delete from Songs;")
cursor.execute("delete from Users;")
cursor.execute("delete from Listens;")
cursor.execute("delete from Recommendations;")

#Insert data into the table

cursor.execute("""
  INSERT INTO Users (user_id, name, email)
  VALUES
    (1, 'Mickey', 'mickey@example.com'),
    (2, 'Minnie', 'minnie@example.com'),
    (3, 'Daffy', 'daffy@example.com'),
    (4, 'Pluto', 'pluto@example.com');
""")
# Insert sample songs from Taylor Swift, Ed Sheeran, Beatles
cursor.execute("""
INSERT INTO Songs (song_id, title, artist, genre)
VALUES
    (1, 'Evermore', 'Taylor Swift', 'Pop'),
    (2, 'Willow', 'Taylor Swift', 'Pop'),
    (3, 'Shape of You', 'Ed Sheeran', 'Rock'),
    (4, 'Photograph', 'Ed Sheeran', 'Rock'),
    (5, 'Shivers', 'Ed Sheeran', 'Rock'),
    (6, 'Yesterday', 'Beatles', 'Classic'),
    (7, 'Yellow Submarine', 'Beatles', 'Classic'),
    (8, 'Hey Jude', 'Beatles', 'Classic'),
    (9, 'Bad Blood', 'Taylor Swift', 'Rock'),
    (10, 'DJ Mix', 'DJ', NULL);
""")

# Insert sample listens
cursor.execute("""
INSERT INTO Listens (listen_id, user_id, song_id, rating, listen_time)
VALUES
    (1, 1, 1, 4.5, '2024-08-30 14:35:00'),
    (2, 1, 2, 4.2, NULL),
    (3, 1, 6, 3.9, '2024-08-29 10:15:00'),
    (4, 2, 2, 4.7, NULL),
    (5, 2, 7, 4.6, '2024-08-28 09:20:00'),
    (6, 2, 8, 3.9, '2024-08-27 16:45:00'),
    (7, 3, 1, 2.9, NULL),
    (8, 3, 2, 4.9, '2024-08-26 12:30:00'),
    (9, 3, 6, NULL, NULL);
""")

# Commit changes and close the connection
conn.commit()
conn.close()

runSql('Users', "select * from Users;")
runSql('Songs', "select * from Songs;")
runSql('Listens', "select * from Listens;")

user_id,name,email
1,Mickey,mickey@example.com
2,Minnie,minnie@example.com
3,Daffy,daffy@example.com
4,Pluto,pluto@example.com


song_id,title,artist,genre
1,Evermore,Taylor Swift,Pop
2,Willow,Taylor Swift,Pop
3,Shape of You,Ed Sheeran,Rock
4,Photograph,Ed Sheeran,Rock
5,Shivers,Ed Sheeran,Rock
6,Yesterday,Beatles,Classic
7,Yellow Submarine,Beatles,Classic
8,Hey Jude,Beatles,Classic
9,Bad Blood,Taylor Swift,Rock
10,DJ Mix,DJ,None


listen_id,user_id,song_id,rating,listen_time
1,1,1,4.5,2024-08-30 14:35:00
2,1,2,4.2,None
3,1,6,3.9,2024-08-29 10:15:00
4,2,2,4.7,None
5,2,7,4.6,2024-08-28 09:20:00
6,2,8,3.9,2024-08-27 16:45:00
7,3,1,2.9,None
8,3,2,4.9,2024-08-26 12:30:00
9,3,6,NaN,None


,listen_id,user_id,song_id,rating,listen_time
0,1,1,1,4.5,2024-08-30 14:35:00
1,2,1,2,4.2,None
2,3,1,6,3.9,2024-08-29 10:15:00
3,4,2,2,4.7,None
4,5,2,7,4.6,2024-08-28 09:20:00
5,6,2,8,3.9,2024-08-27 16:45:00
6,7,3,1,2.9,None
7,8,3,2,4.9,2024-08-26 12:30:00
8,9,3,6,NaN,None


In [298]:
qry_classic_songs = """
SELECT Songs.title, Songs.artist FROM Songs
WHERE Songs.genre= 'Classic';
"""
runSql('Classic Query', qry_classic_songs)

title,artist
Yesterday,Beatles
Yellow Submarine,Beatles
Hey Jude,Beatles


,title,artist
0,Yesterday,Beatles
1,Yellow Submarine,Beatles
2,Hey Jude,Beatles


In [299]:
#Find titles and artists if songs in Classic Genre with titles starting like Ye...

qry_classic_songs2 = """
SELECT Songs.title, Songs.artist FROM Songs
WHERE Songs.genre= 'Classic' AND
  Songs.title LIKE 'Ye%';
"""
runSql('Classic Query with songs like Ye..', qry_classic_songs2)

title,artist
Yesterday,Beatles
Yellow Submarine,Beatles


,title,artist
0,Yesterday,Beatles
1,Yellow Submarine,Beatles


In [300]:
#List all the genres in the Songs table
qry_genres = """
SELECT genre from Songs;
"""
runSql('All genres', qry_genres)

genre
Pop
Pop
Rock
Rock
Rock
Classic
Classic
Classic
Rock
None


,genre
0,Pop
1,Pop
2,Rock
3,Rock
4,Rock
5,Classic
6,Classic
7,Classic
8,Rock
9,None


In [301]:
#Find the unique genres from the Songs table

qry_distinct = """
SELECT DISTINCT genre
FROM Songs;
"""

runSql('Distinct Genres', qry_distinct)

genre
Pop
Rock
Classic
None


,genre
0,Pop
1,Rock
2,Classic
3,None


In [302]:
#Find the number of songs by Taylor Swift in different genres

qry_taylor_count="""
SELECT artist, genre,count(*) as num_songs
FROM Songs
WHERE Songs.artist='Taylor Swift'
GROUP BY artist, genre;
"""
runSql('Count of songs by Taylor Swift in different genres', qry_taylor_count)

artist,genre,num_songs
Taylor Swift,Pop,2
Taylor Swift,Rock,1


,artist,genre,num_songs
0,Taylor Swift,Pop,2
1,Taylor Swift,Rock,1


In [303]:
#Find the number of songs by all artists in different genres

qry_artist_count="""
SELECT artist, genre,count(*) as num_songs
FROM Songs
GROUP BY artist, genre;
"""
runSql('Count of songs by Taylor Swift in different genres', qry_artist_count)


artist,genre,num_songs
Beatles,Classic,3
DJ,None,1
Ed Sheeran,Rock,3
Taylor Swift,Pop,2
Taylor Swift,Rock,1


,artist,genre,num_songs
0,Beatles,Classic,3
1,DJ,None,1
2,Ed Sheeran,Rock,3
3,Taylor Swift,Pop,2
4,Taylor Swift,Rock,1


In [304]:
#Create one single table for songs and listens. Redundancies are allowed

qry_large_table = """
SELECT * FROM Songs
LEFT JOIN Listens ON Songs.song_id=Listens.song_id
LEFT JOIN Users ON Listens.user_id=Users.user_id;
"""
runSql('One Large Table',qry_large_table)

song_id,title,artist,genre,listen_id,user_id,song_id,rating,listen_time,user_id,name,email
1,Evermore,Taylor Swift,Pop,1.0,1.0,1.0,4.5,2024-08-30 14:35:00,1.0,Mickey,mickey@example.com
1,Evermore,Taylor Swift,Pop,7.0,3.0,1.0,2.9,None,3.0,Daffy,daffy@example.com
2,Willow,Taylor Swift,Pop,2.0,1.0,2.0,4.2,None,1.0,Mickey,mickey@example.com
2,Willow,Taylor Swift,Pop,4.0,2.0,2.0,4.7,None,2.0,Minnie,minnie@example.com
2,Willow,Taylor Swift,Pop,8.0,3.0,2.0,4.9,2024-08-26 12:30:00,3.0,Daffy,daffy@example.com
3,Shape of You,Ed Sheeran,Rock,NaN,NaN,NaN,NaN,None,NaN,None,None
4,Photograph,Ed Sheeran,Rock,NaN,NaN,NaN,NaN,None,NaN,None,None
5,Shivers,Ed Sheeran,Rock,NaN,NaN,NaN,NaN,None,NaN,None,None
6,Yesterday,Beatles,Classic,3.0,1.0,6.0,3.9,2024-08-29 10:15:00,1.0,Mickey,mickey@example.com
6,Yesterday,Beatles,Classic,9.0,3.0,6.0,NaN,None,3.0,Daffy,daffy@example.com


,song_id,title,artist,genre,listen_id,user_id,song_id,rating,listen_time,user_id,name,email
0,1,Evermore,Taylor Swift,Pop,1.0,1.0,1.0,4.5,2024-08-30 14:35:00,1.0,Mickey,mickey@example.com
1,1,Evermore,Taylor Swift,Pop,7.0,3.0,1.0,2.9,None,3.0,Daffy,daffy@example.com
2,2,Willow,Taylor Swift,Pop,2.0,1.0,2.0,4.2,None,1.0,Mickey,mickey@example.com
3,2,Willow,Taylor Swift,Pop,4.0,2.0,2.0,4.7,None,2.0,Minnie,minnie@example.com
4,2,Willow,Taylor Swift,Pop,8.0,3.0,2.0,4.9,2024-08-26 12:30:00,3.0,Daffy,daffy@example.com
5,3,Shape of You,Ed Sheeran,Rock,NaN,NaN,NaN,NaN,None,NaN,None,None
6,4,Photograph,Ed Sheeran,Rock,NaN,NaN,NaN,NaN,None,NaN,None,None
7,5,Shivers,Ed Sheeran,Rock,NaN,NaN,NaN,NaN,None,NaN,None,None
8,6,Yesterday,Beatles,Classic,3.0,1.0,6.0,3.9,2024-08-29 10:15:00,1.0,Mickey,mickey@example.com
9,6,Yesterday,Beatles,Classic,9.0,3.0,6.0,NaN,None,3.0,Daffy,daffy@example.com


In [305]:
#Find the Highly Rates Songs. Consider Rating above 4.6 as highly rated

qry_rating_songs="""
SELECT Songs.song_id, Songs.title, Songs.artist, Listens.rating
FROM Songs
JOIN Listens ON Songs.song_id = Listens.song_id
WHERE Listens.rating > 4.6
"""
runSql('Highly Rated Songs',qry_rating_songs)

#Find tha average rating of every song

qry_avg_songs="""
SELECT Songs.song_id, Songs.title, Songs.artist, AVG(Listens.rating)
FROM Songs
JOIN Listens ON Songs.song_id = Listens.song_id
GROUP BY Songs.song_id, Songs.title, Songs.artist
"""
runSql('Highly Rated Songs',qry_avg_songs)



song_id,title,artist,rating
2,Willow,Taylor Swift,4.7
2,Willow,Taylor Swift,4.9


song_id,title,artist,AVG(Listens.rating)
1,Evermore,Taylor Swift,3.7
2,Willow,Taylor Swift,4.6
6,Yesterday,Beatles,3.9
7,Yellow Submarine,Beatles,4.6
8,Hey Jude,Beatles,3.9


,song_id,title,artist,AVG(Listens.rating)
0,1,Evermore,Taylor Swift,3.7
1,2,Willow,Taylor Swift,4.6
2,6,Yesterday,Beatles,3.9
3,7,Yellow Submarine,Beatles,4.6
4,8,Hey Jude,Beatles,3.9


In [306]:
#Find the popular songs by counting the listens

qry_pop_songs = """
SELECT Songs.song_id, Songs.title, Songs.artist, count(Listens.song_id)
FROM Songs
JOIN Listens
ON Songs.song_id=Listens.song_id
GROUP BY Songs.title, Songs.artist
ORDER BY COUNT(Listens.song_id) DESC;
"""

runSql('Popular Songs', qry_pop_songs)

song_id,title,artist,count(Listens.song_id)
2,Willow,Taylor Swift,3
1,Evermore,Taylor Swift,2
6,Yesterday,Beatles,2
8,Hey Jude,Beatles,1
7,Yellow Submarine,Beatles,1


,song_id,title,artist,count(Listens.song_id)
0,2,Willow,Taylor Swift,3
1,1,Evermore,Taylor Swift,2
2,6,Yesterday,Beatles,2
3,8,Hey Jude,Beatles,1
4,7,Yellow Submarine,Beatles,1


In [307]:
#Find songs by Ed Sheeran and Taylor Swift

qry_edtaylor="""
SELECT title, artist
FROM Songs
WHERE artist IN ('Ed Sheeran', 'Taylor Swift');
"""
runSql('Songs by Ed Sheeran and Taylor Swift', qry_edtaylor)

#Find songs from both pop and rock genres

qry_popandrock = """
SELECT title, artist
FROM Songs
WHERE genre='Pop'
UNION
SELECT title, artist
FROM Songs
WHERE genre='Rock'
"""
runSql('Songs on both Pop and Rock', qry_popandrock)


title,artist
Evermore,Taylor Swift
Willow,Taylor Swift
Shape of You,Ed Sheeran
Photograph,Ed Sheeran
Shivers,Ed Sheeran
Bad Blood,Taylor Swift


title,artist
Bad Blood,Taylor Swift
Evermore,Taylor Swift
Photograph,Ed Sheeran
Shape of You,Ed Sheeran
Shivers,Ed Sheeran
Willow,Taylor Swift


,title,artist
0,Bad Blood,Taylor Swift
1,Evermore,Taylor Swift
2,Photograph,Ed Sheeran
3,Shape of You,Ed Sheeran
4,Shivers,Ed Sheeran
5,Willow,Taylor Swift


In [308]:
#Find songs listened to by user_id 1

qry_listen_usr="""
SELECT title, artist
FROM Songs
WHERE song_id IN (SELECT song_id FROM Listens WHERE Listens.listen_time IS NULL);
"""
runSql('Songs listened by user_id=1',qry_listen_usr)

title,artist
Evermore,Taylor Swift
Willow,Taylor Swift
Yesterday,Beatles


,title,artist
0,Evermore,Taylor Swift
1,Willow,Taylor Swift
2,Yesterday,Beatles


In [309]:
#Computing the recommendations
#Find the song pairs which are shared across > 1 user

qry_sharing="""
WITH song_similarity AS (
SELECT u1.song_id as song1, u2.song_id as song2, COUNT(*) AS common_users
FROM LISTENS u1
JOIN LISTENS u2
ON u1.user_id=u2.user_id
AND u1.song_id != u2.song_id
GROUP BY u1.song_id, u2.song_id
HAVING COUNT(*)>1
),

recs AS (
  SELECT user_id, song2 as song_id
  FROM song_similarity
  JOIN Listens as L
  ON L.song_id = song_similarity.song1
  WHERE song_similarity.song2 NOT IN
  (SELECT song_id FROM Listens as temp where temp.user_id=L.user_id)
)
select * from recs;
"""
reco_result = runSql('Songs shared', qry_sharing)

user_id,song_id
2,1
2,6


#ASSIGNMENT-2. To be submitted as a PDF with queries and screenshots.

#1 Insert the above into the recommendations table

#2 Generate the recommendaions for Minnie

#3 Re-do the generation of recommendations now on the basis of listen time

#4 Generate new recommendations

#5 What are the differences with the static method on #2 above

In [310]:
if 'recommendation_id' not in reco_result.columns:
        max_recommendation_id =0
else:
    max_recommendation_id = reco_result['recommendation_id'].max()

reco_result['recommendation_id'] = range(max_recommendation_id + 1, max_recommendation_id + 1 + len(reco_result))
reco_result['recommendation_time'] = datetime.now()
reco_result['recommendation_time'] = reco_result['recommendation_time'].apply(
    lambda x: x.strftime('%Y-%m-%d %H:%M:%S') if pd.notnull(x) else None
)
reco_result = reco_result[['recommendation_id', 'recommendation_time', 'user_id', 'song_id']]

In [314]:
reco_result = reco_result.reset_index()
print(reco_result)

   index  recommendation_id  recommendation_time  user_id  song_id
0      0                  1  2024-12-06 19:58:06        2        1
1      1                  2  2024-12-06 19:58:06        2        6


In [315]:
conn = sqlite3.connect(dbname)
cursor = conn.cursor()
cursor.execute("PRAGMA table_info(recommendations)")
print(cursor.fetchall())
conn.close()


[(0, 'recommendation_id', 'INTEGER', 1, None, 0), (1, 'recommendation_time', 'TIMESTAMP', 0, None, 0), (2, 'user_id', 'INTEGER', 1, None, 0), (3, 'song_id', 'INTEGER', 1, None, 0)]


In [316]:
conn = sqlite3.connect(dbname)
reco_result.to_sql('recommendations', conn,if_exists='append', index=False)
conn.close()

OperationalError: table "recommendations" already exists

In [ ]:
# write_result = write_to_table(reco_result,'recommendations')
# print(write_result)

False


In [ ]:
test = "Select * from Recommendations "
result = runSql('Testing',test)
print(result)

recommendation_id,recommendation_time,user_id,song_id


Empty DataFrame
Columns: [recommendation_id, recommendation_time, user_id, song_id]
Index: []
